In [1]:
import com.google.gson.Gson
import hu.raven.puppet.job.CostStatistics
import java.io.File

val inputFolder = File("D:\\Git\\GitHub\\SourceCodes\\Kotlin\\BacterialMemeticConsole\\output")
val costStatistics = inputFolder.listFiles()
    .map { file ->
        println(file)
        Gson().fromJson<CostStatistics>(file.readLines().joinToString("\n"), CostStatistics::class.java)
    }

D:\Git\GitHub\SourceCodes\Kotlin\BacterialMemeticConsole\output\withStandardDeviation0_01.json
D:\Git\GitHub\SourceCodes\Kotlin\BacterialMemeticConsole\output\withStandardDeviation0_02.json
D:\Git\GitHub\SourceCodes\Kotlin\BacterialMemeticConsole\output\withStandardDeviation0_04.json
D:\Git\GitHub\SourceCodes\Kotlin\BacterialMemeticConsole\output\withStandardDeviation0_08.json
D:\Git\GitHub\SourceCodes\Kotlin\BacterialMemeticConsole\output\withStandardDeviation0_16.json
D:\Git\GitHub\SourceCodes\Kotlin\BacterialMemeticConsole\output\withStandardDeviation0_32.json


In [2]:
val initialGapStatistics = costStatistics.map { it.initialGap }
val builtGapStatistics = costStatistics.map { it.builtGap }
val optimizedGapStatistics = costStatistics.map { it.optimizedGap }

val data = mapOf(
    "phase" to List(6) { "initial"} + List(6) { "local search"} +List(6) { "final"} ,
    "Q1" to initialGapStatistics.map { it.q1 * 100 } + builtGapStatistics.map { it.q1 * 100} + optimizedGapStatistics.map { it.q1 * 100},
    "median" to initialGapStatistics.map { it.median * 100} + builtGapStatistics.map { it.median* 100 } + optimizedGapStatistics.map { it.median * 100},
    "Q3" to initialGapStatistics.map { it.q2 * 100} + builtGapStatistics.map { it.q2* 100 } + optimizedGapStatistics.map { it.q2* 100 },
)

In [3]:
%use kandy

In [5]:
import org.jetbrains.letsPlot.scale.guideLegend

data.plot {
    layout {
        title = "Gap of new solution in the three phases"
        guideLegend()
    }

    x(
        listOf("0.01", "0.02", "0.04", "0.08", "0.16", "0.32") +
                listOf("0.01", "0.02", "0.04", "0.08", "0.16", "0.32") +
                listOf("0.01", "0.02", "0.04", "0.08", "0.16", "0.32")
    ) { axis.name = "noise deviation" }
    y {
        axis.name = "gap%"
        scale = continuous(0.0..70.0)
    }

    groupBy("phase") {
        ribbon {
            yMin("Q1")
            yMax("Q3")
            fillColor = Color.BLACK
            alpha = 0.25
            borderLine {
                borderLine.color = Color.GREEN
                width = 0.0
            }
        }

        line {
            y("median")
            color("phase") {
                scale = categorical("final" to Color.BLACK, "local search" to Color.BLACK, "initial" to Color.BLACK)
            }
            width = 1.5
            type("phase") {
                scale = categorical(
                    "final" to LineType.DASHED, 
                    "local search" to LineType.DOTTED,
                    "initial" to LineType.TWODASH
                )
                color= Color.BLACK 
            }
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="b7JtfV"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Gap of new solution in the three phases"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"noise deviation"
},{
"aesthetic":"x",
"discrete":true,
"name":"noise deviation"
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#000000","#000000","#000000"],
"limits":["final","local search","initial"]
},{
"aesthetic":"linetype",
"values":[2.0,3.0,6.0],
"limits":["final","local search","initial"]
},{
"aesthetic":"y",
"name":"gap%",
"limits":[0.0,70.0]
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"Q1",
"ymax":"Q3",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"Q1":[5.7192914341769985,5.677374239922828,5.49919691950671,5.7839802076369295,7.1944710317741745,58.37799186589186,2.575417553716841,3.013549605750465,2.65269411086384,2.766477576361437,4.779263077860518,47.62498823958212,0.2328533608521921,0.5853830986623842,0.7348456910146739,0.34832005143237144,0.7794708950809914,2.316975276635125],
"Q3":[10.89973743497541,11.848523272454937,10.287101188154368,11.70116632467808,15.120226329394981,76.76750195620758,6.722731638262314,7.6149830865319235,6.828304659863482,7.634407648284358,11.152236924700576,64.08301038429975,2.6167853888172887,2.6121509986405833,2.508467456087371,2.89129884766659,3.014655182587078,4.522428003107026],
"&merged_groups":["initial","initial","initial","initial","initial","initial","local search","local search","local search","local search","local search","local search","final","final","final","final","final","final"],
"x":["0.01","0.02","0.04","0.08","0.16","0.32","0.01","0.02","0.04","0.08","0.16","0.32","0.01","0.02","0.04","0.08","0.16","0.32"]
},
"color":"#3ba272",
"size":0.0,
"sampling":"none",
"alpha":0.25,
"position":"identity",
"geom":"ribbon",
"fill":"#000000"
},{
"mapping":{
"x":"x",
"y":"median",
"color":"phase",
"linetype":"phase",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"phase":["initial","initial","initial","initial","initial","initial","local search","local search","local search","local search","local search","local search","final","final","final","final","final","final"],
"&merged_groups":["initial","initial","initial","initial","initial","initial","local search","local search","local search","local search","local search","local search","final","final","final","final","final","final"],
"median":[7.538928086422914,8.820887632617215,7.4888548088424045,8.839094503832934,10.769957292754029,66.37292135245751,4.11409569479857,4.370643522363737,4.853695771870736,5.169077788051157,7.576701604477054,53.84735895882968,1.2521897176027474,1.4893909475229838,1.4671099235614493,1.3660473926732308,1.7027505971184187,3.1907649100396007],
"x":["0.01","0.02","0.04","0.08","0.16","0.32","0.01","0.02","0.04","0.08","0.16","0.32","0.01","0.02","0.04","0.08","0.16","0.32"]
},
"size":1.5,
"color":"#000000",
"sampling":"none",
"position":"identity",
"geom":"line"
}]
};
 var plotContainer = document.getElementById("b7JtfV");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M15.218877091859401 278.38315771581557 L15.218877091859401 278.38315771581557 L91.3132625511564 274.66351206783446 L167.4076480104534 277.94304161277694 L243.50203346975042 274.58253434936006 L319.5964189290474 259.9172980152093 L395.69080438834436 39.25783982649068 L395.69080438834436 107.86855552070313 L319.5964189290474 286.485150026062 L243.50203346975042 294.8761129608829 L167.4076480104534 2

In [6]:
import hu.akos.hollo.szabo.collections.asImmutable
import hu.akos.hollo.szabo.math.asDoubleVector
import hu.akos.hollo.szabo.math.matrix.DoubleMatrix

var matrix = File("D:\\Git\\GitHub\\SourceCodes\\Kotlin\\BacterialMemeticConsole\\array\\array_data.txt").readLines()
    .map {
        it
            .split(" ")
            .map { it.toDouble() }
            .toDoubleArray()
            .asDoubleVector()
    }
    .toTypedArray()
    .asImmutable()
    .let { DoubleMatrix(it) }

In [7]:
matrix = matrix.indices.asList().last()
    .map {
        matrix.getRow(it)
    }
    .toTypedArray()
    .asImmutable()
    .let { DoubleMatrix(it) }

In [8]:
matrix.dimensions

[18, 100]

In [10]:
import hu.raven.puppet.job.experiments.statistics

val statistics = matrix.columnVectors.asList()
    .map { it.asDoubleArray().statistics() }

In [11]:
statistics.size

18

In [12]:
val newData = mapOf(
    "phase" to List(6) { listOf("final","local search","initial")}.flatten() ,
    "Q1" to statistics.map { it.q1  } ,
    "median" to statistics.map { it.median } ,
    "Q3" to statistics.map { it.q2 },
)

In [13]:
newData.plot {
    layout {
        title = "Gap of original solution in the three phases"
    }

    x(
        listOf("0.01", "0.02", "0.04", "0.08", "0.16", "0.32") +
                listOf("0.01", "0.02", "0.04", "0.08", "0.16", "0.32") +
                listOf("0.01", "0.02", "0.04", "0.08", "0.16", "0.32")
    ) { axis.name = "noise deviation" }
    y {
        axis.name = "gap%"
        axis.breaks(List(9/4*5 + 2){it*20})
    }

    groupBy("phase") {
        ribbon {
            yMin("Q1")
            yMax("Q3")
            fillColor = Color.BLACK
            alpha = 0.25
            borderLine {
                borderLine.color = Color.GREEN
                width = 0.0
            }
        }

        line {
            y("median")
            color("phase") {
                scale = categorical("final" to Color.BLACK, "local search" to Color.BLACK, "initial" to Color.BLACK)
            }
            width = 1.5
            type("phase") {
                scale = categorical(
                    "final" to LineType.DASHED,
                    "local search" to LineType.DOTTED,
                    "initial" to LineType.TWODASH
                )
                color= Color.BLACK
            }
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="rqqOTg"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Gap of original solution in the three phases"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"noise deviation"
},{
"aesthetic":"x",
"discrete":true,
"name":"noise deviation"
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"values":["#000000","#000000","#000000"],
"limits":["final","local search","initial"]
},{
"aesthetic":"linetype",
"values":[2.0,3.0,6.0],
"limits":["final","local search","initial"]
},{
"aesthetic":"y",
"breaks":[0.0,20.0,40.0,60.0,80.0,100.0,120.0,140.0,160.0,180.0,200.0,220.0],
"name":"gap%",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"Q1",
"ymax":"Q3",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"Q1":[191.99737097205008,192.07959775168018,192.7285277926381,190.42655929865484,193.48494531299005,194.34043326445288,25.694734218381043,26.726648993127135,27.25704871002479,27.957806502342407,27.781151098224278,26.691062014776357,14.364488468764014,13.475077166849037,14.288094583795296,13.028809755487035,14.355578183354268,13.525908157664702],
"&merged_groups":["final","final","final","final","final","final","local search","local search","local search","local search","local search","local search","initial","initial","initial","initial","initial","initial"],
"Q3":[218.80285670115893,214.67167581175852,220.16987831131672,219.07106636290146,215.6573978653039,219.56448107298013,35.11268007416917,35.01641599130456,35.509351332590185,36.119929744126786,35.53278731366687,35.47986116917916,17.731567716276086,17.48432106223263,17.927295906651896,17.841658652338445,18.916799891279744,17.44964066488972],
"x":["0.01","0.02","0.04","0.08","0.16","0.32","0.01","0.02","0.04","0.08","0.16","0.32","0.01","0.02","0.04","0.08","0.16","0.32"]
},
"color":"#3ba272",
"size":0.0,
"sampling":"none",
"alpha":0.25,
"position":"identity",
"geom":"ribbon",
"fill":"#000000"
},{
"mapping":{
"x":"x",
"y":"median",
"color":"phase",
"linetype":"phase",
"group":"&merged_groups"
},
"stat":"identity",
"data":{
"phase":["final","final","final","final","final","final","local search","local search","local search","local search","local search","local search","initial","initial","initial","initial","initial","initial"],
"&merged_groups":["final","final","final","final","final","final","local search","local search","local search","local search","local search","local search","initial","initial","initial","initial","initial","initial"],
"median":[207.2095287620417,205.1549294196415,207.8685972325086,206.25695076587095,206.77863219540598,205.19804856917966,31.77397737012557,30.91770965297913,31.119407474919967,31.492619291544056,30.790601593744693,31.97891408738649,15.942611476206082,15.427879063638517,15.910952432126034,15.673674059163712,17.32109581986596,15.346961113826074],
"x":["0.01","0.02","0.04","0.08","0.16","0.32","0.01","0.02","0.04","0.08","0.16","0.32","0.01","0.02","0.04","0.08","0.16","0.32"]
},
"size":1.5,
"color":"#000000",
"sampling":"none",
"position":"identity",
"geom":"line"
}]
};
 var plotContainer = document.getElementById("rqqOTg");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M14.439814083700831 16.402790733584823 L14.439814083700831 16.402790733584823 L86.63888450220497 21.878352173475434 L158.83795492070914 14.590909090909179 L231.03702533921327 16.047299526358472 L303.2360957577174 20.571853665823255 L375.43516617622157 15.393316408041471 L375.43516617622157